# [Module 2.1] Create Dataset Group

MovieLens 데이터 세트에서 수집된 데이터를 기반으로, 영화에 대한 추천 모델을 작성하는 법을 안내합니다. 목표는 특정 사용자를 기반으로 하는 영화를 추천하는 것입니다.<br>
이 노트북에서는 전체적으로 데이타를 준비하는 단계입니다. 그러기 위해서 아래와 같은 작업을 수행 합니다. <br>

* S3 버킷 설정, 데이타 다운로드 및 S3에 데이타 업로드
* 데이타 스키마 생성
* 데이타 세트 그룹 생성 (DatasetGroup)
* 데이타 세트 생성 (Dataset)
* 환경 설정 (S3 버킷에 정책(Policy) 부여, Personalize 역할(Role) 생성)
* 데이타 Import (S3 --> Personalize 서비스로 이동)

이 노트북을 모두 실행하는데 걸리는 시간은 약 20 ~ 30 분 소요 됩니다.


In [1]:
# Imports
import boto3
import json
import numpy as np
import pandas as pd
import time
from datetime import datetime

import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter
import matplotlib.dates as mdate
from botocore.exceptions import ClientError

다음으로 여러분의 환경이 Amazon Personalize와 성공적으로 통신할 수 있는지 확인해야 합니다.

In [2]:
# Configure the SDK to Personalize:
personalize = boto3.client('personalize')

생성할 오브젝트의 끝에 임의의 숫자를 부여하기 위해 suffix 정의

In [3]:
suffix = str(np.random.uniform())[4:9]

In [4]:
%store -r

## Personalize IAM Role 생성

또한, Amazon Personalize는 특정 작업들을 실행할 권한을 갖기 위해, AWS에서 역할을 맡을 수 있는 기능이 필요합니다. 

In [5]:
iam = boto3.client("iam")

# Personalize 서비스가 이용할 role을 만들기 위한 assume_role_policy 생성
role_name = "PersonalizeRoleDemo" + suffix
assume_role_policy_document = {
    "Version": "2012-10-17",
    "Statement": [
        {
          "Effect": "Allow",
          "Principal": {
            "Service": "personalize.amazonaws.com"
          },
          "Action": "sts:AssumeRole"
        }
    ]
}

# Personalize 서비스가 이용할 role 생성
create_role_response = iam.create_role(
    RoleName = role_name,
    AssumeRolePolicyDocument = json.dumps(assume_role_policy_document)
)

# 위에서 생성한 role에 AmazonPersonalizeFullAccess 권한 추가
policy_arn = "arn:aws:iam::aws:policy/service-role/AmazonPersonalizeFullAccess"
iam.attach_role_policy(
    RoleName = role_name,
    PolicyArn = policy_arn
)

# 위에서 생성한 role에 AmazonS3FullAccess 권한 추가
iam.attach_role_policy(
    RoleName=role_name,    
    PolicyArn='arn:aws:iam::aws:policy/AmazonS3FullAccess'
)
time.sleep(60) # wait for a minute to allow IAM role policy attachment to propagate

role_arn = create_role_response["Role"]["Arn"]
print(role_arn)

arn:aws:iam::057716757052:role/PersonalizeRoleDemo91891


## 데이터 세트 그룹 생성 및 대기

Personalize에서 가장 큰 단위는 **데이터 세트 그룹(Dataset Group)** 이며, 이렇게 하면 데이터, 이벤트 추적기(event tracker), 솔루션(solution) 및 캠페인(campaign)이 분리됩니다. 공통의 데이터 수집을 공유하는 것들을 그룹화합니다. 원하는 경우 아래 그룹명을 자유롭게 변경해 주세요.

### 데이터 세트 그룹 생성

In [6]:
create_dataset_group_response = personalize.create_dataset_group(
    name = "Movielens-dataset-group-" + suffix
)

dataset_group_arn = create_dataset_group_response['datasetGroupArn']
print(json.dumps(create_dataset_group_response, indent=2))

{
  "datasetGroupArn": "arn:aws:personalize:ap-northeast-2:057716757052:dataset-group/Movielens-dataset-group-91891",
  "ResponseMetadata": {
    "RequestId": "59cb472a-44d8-43da-bb31-ec5c22e27fe8",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Sun, 30 Aug 2020 07:27:32 GMT",
      "x-amzn-requestid": "59cb472a-44d8-43da-bb31-ec5c22e27fe8",
      "content-length": "113",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


#### 데이터 세트 그룹이 활성화 상태가 될 때까지 대기

아래의 모든 항목에서 Dataset Group을 사용하려면 활성화(active)가 되어야 합니다. 아래 셀을 실행하고 DatasetGroup: ACTIVE로 변경될 때까지 기다려 주세요.

In [7]:
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_dataset_group_response = personalize.describe_dataset_group(
        datasetGroupArn = dataset_group_arn
    )
    status = describe_dataset_group_response["datasetGroup"]["status"]
    print("DatasetGroup: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(15)

DatasetGroup: ACTIVE


## 스키마 생성

Personalize가 데이터를 이해하는 방법의 핵심 구성 요소는 아래 정의 된 스키마(schema)에서 비롯됩니다. 이 설정은 CSV 파일을 통해 제공된 데이터를 요약하는 방법을 Personalize 서비스에 알려줍니다. 열(column)과 유형(type)은 위에서 만든 파일의 내용과 일치합니다.

In [8]:
interaction_schema_name="DEMO-interaction-schema-"+suffix

schema = {
    "type": "record",
    "name": "Interactions",
    "namespace": "com.amazonaws.personalize.schema",
    "fields": [
        {
            "name": "USER_ID",
            "type": "string"
        },
        {
            "name": "ITEM_ID",
            "type": "string"
        },
        {
            "name": "EVENT_VALUE",
            "type": "float"
        },
        {
            "name": "TIMESTAMP",
            "type": "long"
        },
        { 
            "name": "EVENT_TYPE",
            "type": "string"
        },
    ],
    "version": "1.0"
}


create_schema_response = personalize.create_schema(
    name = interaction_schema_name,
    schema = json.dumps(schema)
)

interaction_schema_arn = create_schema_response['schemaArn']
print(json.dumps(create_schema_response, indent=2))

{
  "schemaArn": "arn:aws:personalize:ap-northeast-2:057716757052:schema/DEMO-interaction-schema-91891",
  "ResponseMetadata": {
    "RequestId": "696db3cc-e680-46d4-9542-51b5b632fb91",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Sun, 30 Aug 2020 07:27:42 GMT",
      "x-amzn-requestid": "696db3cc-e680-46d4-9542-51b5b632fb91",
      "content-length": "100",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


In [9]:
item_schema_name="DEMO-item-schema-"+suffix

item_schema = {
    "type": "record",
    "name": "Items",
    "namespace": "com.amazonaws.personalize.schema",
    "fields": [
    {
        "name": "ITEM_ID",
        "type": "string"
    },
    {
        "name": "GENRE",
        "type": "string",
        "categorical": True
    }
    ],
    "version": "1.0"
}

create_metadata_schema_response = personalize.create_schema(
    name = item_schema_name,
    schema = json.dumps(item_schema)
)

item_schema_arn = create_metadata_schema_response['schemaArn']
print(json.dumps(create_metadata_schema_response, indent=2))

{
  "schemaArn": "arn:aws:personalize:ap-northeast-2:057716757052:schema/DEMO-item-schema-91891",
  "ResponseMetadata": {
    "RequestId": "cd3a397a-55ff-4ab7-ba5a-66de339482ce",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Sun, 30 Aug 2020 07:27:45 GMT",
      "x-amzn-requestid": "cd3a397a-55ff-4ab7-ba5a-66de339482ce",
      "content-length": "93",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


## 데이터 세트 생성

그룹 다음으로 생성할 것은 실제 데이터 세트입니다. 아래의 코드 셀을 실행하여 데이터 세트을 생성해 주세요.

### Interaction 데이터 세트 생성

In [10]:
dataset_type = "INTERACTIONS"
create_dataset_response = personalize.create_dataset(
    name = "DEMO-interaction-dataset-" + suffix,
    datasetType = dataset_type,
    datasetGroupArn = dataset_group_arn,
    schemaArn = interaction_schema_arn
)

interaction_dataset_arn = create_dataset_response['datasetArn']
print(json.dumps(create_dataset_response, indent=2))

{
  "datasetArn": "arn:aws:personalize:ap-northeast-2:057716757052:dataset/Movielens-dataset-group-91891/INTERACTIONS",
  "ResponseMetadata": {
    "RequestId": "3a99ea97-3994-428d-a484-824db154c112",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Sun, 30 Aug 2020 07:27:48 GMT",
      "x-amzn-requestid": "3a99ea97-3994-428d-a484-824db154c112",
      "content-length": "115",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


###  ITEM 데이터 세트 생성 

In [11]:
dataset_type = "ITEMS"
create_item_dataset_response = personalize.create_dataset(
    name = "DEMO-item-dataset-" + suffix,
    datasetType = dataset_type,
    datasetGroupArn = dataset_group_arn,
    schemaArn = item_schema_arn,
  
)

item_dataset_arn = create_item_dataset_response['datasetArn']
print(json.dumps(create_item_dataset_response, indent=2))

{
  "datasetArn": "arn:aws:personalize:ap-northeast-2:057716757052:dataset/Movielens-dataset-group-91891/ITEMS",
  "ResponseMetadata": {
    "RequestId": "9c607a75-587e-4185-a082-a591a0383f14",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Sun, 30 Aug 2020 07:27:51 GMT",
      "x-amzn-requestid": "9c607a75-587e-4185-a082-a591a0383f14",
      "content-length": "108",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


### 데이터 세트 Import

이전에는 정보를 저장하기 위해 데이터 세트 그룹 및 데이터 세트를 생성했으므로, 
이제는 모델 구축을 위해 S3에서 Amazon Personalize로 데이터를 로드하는 import job을 실행합니다.

#### Interaction 데이터 세트 Import Job 생성

In [12]:
create_dataset_import_job_response = personalize.create_dataset_import_job(
    jobName = "demo-interaction-dataset-import-" + suffix,
    datasetArn = interaction_dataset_arn,
    dataSource = {
        "dataLocation": "s3://{}/{}".format(bucket, interaction_filename)
    },
    roleArn = role_arn
)

interation_dataset_import_job_arn = create_dataset_import_job_response['datasetImportJobArn']
print(json.dumps(create_dataset_import_job_response, indent=2))

{
  "datasetImportJobArn": "arn:aws:personalize:ap-northeast-2:057716757052:dataset-import-job/demo-interaction-dataset-import-91891",
  "ResponseMetadata": {
    "RequestId": "714e8703-7d97-4a0b-af5e-348fa039c8a9",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Sun, 30 Aug 2020 07:27:58 GMT",
      "x-amzn-requestid": "714e8703-7d97-4a0b-af5e-348fa039c8a9",
      "content-length": "130",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


#### 아이템 데이터 세트 Import Job 생성

In [13]:
create_item_dataset_import_job_response = personalize.create_dataset_import_job(
    jobName = "demo-item-dataset-import-" + suffix,
    datasetArn = item_dataset_arn,
    dataSource = {
        "dataLocation": "s3://{}/{}".format(bucket, items_filename)
    },
    roleArn = role_arn
)

item_dataset_import_job_arn = create_dataset_import_job_response['datasetImportJobArn']
print(json.dumps(create_dataset_import_job_response, indent=2))


{
  "datasetImportJobArn": "arn:aws:personalize:ap-northeast-2:057716757052:dataset-import-job/demo-interaction-dataset-import-91891",
  "ResponseMetadata": {
    "RequestId": "714e8703-7d97-4a0b-af5e-348fa039c8a9",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Sun, 30 Aug 2020 07:27:58 GMT",
      "x-amzn-requestid": "714e8703-7d97-4a0b-af5e-348fa039c8a9",
      "content-length": "130",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


### 데이터 세트 Import job이 활성화 상태가 될 때까지 대기

Import job이 완료되기까지 시간이 걸립니다. 아래 코드 셀의 출력 결과가 DatasetImportJob: ACTIVE가 될 때까지 기다려 주세요.

In [14]:
%%time

status = None
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_dataset_import_job_response = personalize.describe_dataset_import_job(
        datasetImportJobArn = interation_dataset_import_job_arn
    )
    
    dataset_import_job = describe_dataset_import_job_response["datasetImportJob"]
    if "latestDatasetImportJobRun" not in dataset_import_job:
        status = dataset_import_job["status"]
        print("DatasetImportJob: {}".format(status))
    else:
        status = dataset_import_job["latestDatasetImportJobRun"]["status"]
        print("LatestDatasetImportJobRun: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

DatasetImportJob: CREATE PENDING
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: ACTIVE
CPU times: user 59.9 ms, sys: 19.5 ms, total: 79.4 ms
Wall time: 16min 1s


In [15]:
status = None
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_dataset_import_job_response = personalize.describe_dataset_import_job(
        datasetImportJobArn = item_dataset_import_job_arn
    )
    
    dataset_import_job = describe_dataset_import_job_response["datasetImportJob"]
    if "latestDatasetImportJobRun" not in dataset_import_job:
        status = dataset_import_job["status"]
        print("DatasetImportJob: {}".format(status))
    else:
        status = dataset_import_job["latestDatasetImportJobRun"]["status"]
        print("LatestDatasetImportJobRun: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)



DatasetImportJob: ACTIVE


## Review

위의 코드를 사용하여 데이타 세트 그룹, 데이타 세트, 데이타 세트 Import까지를 수행 하였습니다. 다음 단계는 이를 기반으로 솔류션(모델)을 생성하는 단계를 진행 합니다..

이제 다음 노트북으로 넘어갈 준비가 되었습니다. (`2.Creating_and_Evaluating_Solutions.ipynb`)


## 다음 노트북에 대한 참고 사항

다음 실습에 필요한 몇 가지 값들이 있습니다. 아래 셀을 실행하여 저장한 후, 다음 주피터 노트북에서 그대로 사용할 수 있습니다.

In [18]:
%store dataset_group_arn
%store interaction_dataset_arn
%store item_dataset_arn
%store interaction_schema_arn
%store item_schema_arn
%store role_name
%store role_arn
%store suffix

Stored 'dataset_group_arn' (str)
Stored 'interaction_dataset_arn' (str)
Stored 'item_dataset_arn' (str)
Stored 'interaction_schema_arn' (str)
Stored 'item_schema_arn' (str)
Stored 'role_name' (str)
Stored 'role_arn' (str)
Stored 'suffix' (str)
